In [5]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [6]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [7]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [8]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-03-01T03:14:27.690Z,38.826500,-122.853836,1.800000,1.60,md,33.0,61.0,0.002592,0.03,...,2022-03-01T03:23:10.404Z,"10km WNW of The Geysers, CA",earthquake,0.18,0.31,0.040,8.0,automatic,nc,nc
1,2022-03-01T03:13:29.825Z,63.343400,-144.877100,4.000000,1.60,ml,NaN,NaN,NaN,0.78,...,2022-03-01T03:18:43.159Z,"36 km SSW of Dry Creek, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
2,2022-03-01T03:08:06.070Z,19.232500,-155.369507,31.250000,1.95,md,24.0,198.0,NaN,0.14,...,2022-03-01T03:11:20.760Z,"11 km ENE of Pāhala, Hawaii",earthquake,1.11,0.94,1.950,4.0,automatic,hv,hv
3,2022-03-01T03:07:10.220Z,38.834167,-122.806663,2.280000,1.30,md,26.0,49.0,0.011080,0.02,...,2022-03-01T03:23:22.405Z,"7km W of Cobb, CA",earthquake,0.20,0.40,0.110,6.0,automatic,nc,nc
4,2022-03-01T03:06:56.000Z,19.153000,-155.472672,32.380001,1.70,md,37.0,151.0,NaN,0.15,...,2022-03-01T03:10:10.760Z,"5 km S of Pāhala, Hawaii",earthquake,0.52,0.84,1.540,6.0,automatic,hv,hv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10530,2022-01-30T03:46:53.207Z,63.278200,-151.251700,7.400000,1.00,ml,NaN,NaN,NaN,0.66,...,2022-02-25T00:07:27.831Z,"37 km SE of Denali National Park, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
10531,2022-01-30T03:44:08.272Z,67.671500,-160.829900,6.200000,2.90,ml,NaN,NaN,NaN,0.57,...,2022-02-25T03:00:49.040Z,"79 km NNW of Kiana, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
10532,2022-01-30T03:37:46.570Z,19.221167,-155.389160,31.940001,1.83,md,43.0,152.0,NaN,0.13,...,2022-01-30T03:40:57.760Z,"9 km ENE of Pāhala, Hawaii",earthquake,0.59,1.16,0.620,5.0,automatic,hv,hv
10533,2022-01-30T03:32:41.102Z,2.757300,128.234600,53.450000,4.50,mb,NaN,194.0,2.154000,1.03,...,2022-02-05T01:40:12.040Z,"Halmahera, Indonesia",earthquake,12.30,10.40,0.272,5.0,reviewed,us,us


In [9]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

Load from database to update with new rows and remove duplicate rows

In [10]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.10750,1.100000,1.30,ml,8.0,149.46,0.875,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.81800,7.300000,1.40,ml,11.0,154.64,0.119,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.88520,30.000000,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.76250,111.300000,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.76570,1.900000,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30715,2022-01-30T03:46:53.207Z,63.278200,-151.25170,7.400000,1.00,ml,NaN,NaN,NaN,0.6600,...,2022-02-25T00:07:27.831Z,"37 km SE of Denali National Park, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
30716,2022-01-30T03:44:08.272Z,67.671500,-160.82990,6.200000,2.90,ml,NaN,NaN,NaN,0.5700,...,2022-02-25T03:00:49.040Z,"79 km NNW of Kiana, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
30717,2022-01-30T03:37:46.570Z,19.221167,-155.38916,31.940001,1.83,md,43.0,152.00,NaN,0.1300,...,2022-01-30T03:40:57.760Z,"9 km ENE of Pāhala, Hawaii",earthquake,0.59,1.16,0.620,5.0,automatic,hv,hv
30718,2022-01-30T03:32:41.102Z,2.757300,128.23460,53.450000,4.50,mb,NaN,194.00,2.154,1.0300,...,2022-02-05T01:40:12.040Z,"Halmahera, Indonesia",earthquake,12.30,10.40,0.272,5.0,reviewed,us,us


In [11]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.10750,1.100000,1.30,ml,8.0,149.46,0.875,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.81800,7.300000,1.40,ml,11.0,154.64,0.119,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.88520,30.000000,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.76250,111.300000,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.76570,1.900000,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28330,2022-01-30T03:46:53.207Z,63.278200,-151.25170,7.400000,1.00,ml,NaN,NaN,NaN,0.6600,...,2022-02-25T00:07:27.831Z,"37 km SE of Denali National Park, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
28331,2022-01-30T03:44:08.272Z,67.671500,-160.82990,6.200000,2.90,ml,NaN,NaN,NaN,0.5700,...,2022-02-25T03:00:49.040Z,"79 km NNW of Kiana, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
28332,2022-01-30T03:37:46.570Z,19.221167,-155.38916,31.940001,1.83,md,43.0,152.00,NaN,0.1300,...,2022-01-30T03:40:57.760Z,"9 km ENE of Pāhala, Hawaii",earthquake,0.59,1.16,0.620,5.0,automatic,hv,hv
28333,2022-01-30T03:32:41.102Z,2.757300,128.23460,53.450000,4.50,mb,NaN,194.00,2.154,1.0300,...,2022-02-05T01:40:12.040Z,"Halmahera, Indonesia",earthquake,12.30,10.40,0.272,5.0,reviewed,us,us


In [12]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [13]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

False


In [14]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
30692  2022-01-30T05:22:23.246Z   37.8445   144.8923   10.00  4.3      mb   
30693  2022-01-30T05:08:20.279Z  -30.0533  -177.1517   10.00  4.3      mb   
30714  2022-01-30T03:50:17.124Z   54.2336  -159.7370   22.54  3.0      ml   
30715  2022-01-30T03:46:53.207Z   63.2782  -151.2517    7.40  1.0      ml   
30719  2022-01-30T03:31:21.676Z   54.5942  -159.7845   19.20  2.3      ml   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [15]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude  longitude       depth   mag  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.10750    1.100000  1.30   
1      2022-01-06T01:31:33.698Z  38.181000 -117.81800    7.300000  1.40   
2      2022-01-06T01:24:20.083Z  61.340700 -149.88520   30.000000  1.50   
3      2022-01-06T01:15:15.851Z  62.870300 -150.76250  111.300000  2.10   
4      2022-01-06T01:10:44.629Z  64.014500 -148.76570    1.900000  1.50   
...                         ...        ...        ...         ...   ...   
28330  2022-01-30T03:46:53.207Z  63.278200 -151.25170    7.400000  1.00   
28331  2022-01-30T03:44:08.272Z  67.671500 -160.82990    6.200000  2.90   
28332  2022-01-30T03:37:46.570Z  19.221167 -155.38916   31.940001  1.83   
28333  2022-01-30T03:32:41.102Z   2.757300  128.23460   53.450000  4.50   
28334  2022-01-30T03:31:21.676Z  54.594200 -159.78450   19.200000  2.30   

      magType   nst     gap   dmin     rms  ...                   updated  \
0          ml   8.0  1

In [16]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

In [17]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

In [18]:
#copy data as csv as backup method
model_df.to_csv("earthquake_raw.csv",index=False)

In [19]:
# Close connection to SQLite database
conn.close()
connection.close()